**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_combeffect 
BASE DIRECTORY:     /data/reddylab/Kuei 
WORK DIRECTORY:     /data/reddylab/Kuei/out 
CODE DIRECTORY:     /data/reddylab/Kuei/code 
PATH OF SOURCE:     /data/reddylab/Kuei/source 
PATH OF EXECUTABLE: /data/reddylab/Kuei/bin 
PATH OF ANNOTATION: /data/reddylab/Kuei/annotation 
PATH OF PROJECT:    /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS:    /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 


In [5]:
fdiry = file.path(FD_RES, "region", "KS91_K562_ASTARRseq_peak_macs")
fname = "KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed"
fpath = file.path(fdiry, fname)

cnames = c("Chrom", "Start", "End")
dat = read_tsv(fpath, col_names = cnames, show_col_types = FALSE)

dat_peak_wgs = dat %>% mutate(Peak = paste(Chrom, Start, End, sep="_"))
print(dim(dat))
head(dat)

[1] 247520      3


Chrom,Start,End
<chr>,<dbl>,<dbl>
chr1,10015,10442
chr1,14253,14645
chr1,16015,16477
chr1,17237,17772
chr1,28903,29613
chr1,30803,31072


In [6]:
CHROMS_WGS = paste0("chr", c(1:22, "X", "Y"))
REGIONS=c("GATA1",    "MYC",       "FADS")
CHROMS=c( "chrX",     "chr8",      "chr11")
STARTS=c( 47785501, 126735901, 61554801)
ENDS=c(   49880650, 128736550, 61665820)

In [16]:
lst = lapply(seq_along(REGIONS), function(idx){
    REGION = REGIONS[idx]
    x_chrom  = CHROMS[idx]
    x_start  = STARTS[idx]
    x_end    = ENDS[idx]
    
    dat = dat_peak_wgs
    dat = dat %>% 
        dplyr::filter(Chrom == x_chrom, End >= x_start, Start <= x_end) %>%
        dplyr::mutate(Region = REGION)
    
    return(dat)
})

dat = bind_rows(lst) %>% 
    mutate(Chrom = factor(Chrom, levels=CHROMS_WGS),
           Start = as.integer(Start),
           End   = as.integer(End)) %>%
    arrange(Chrom, Start, End) %>%
    distinct

dat_peak = dat
print(dim(dat))
head(dat)

[1] 417   5


Chrom,Start,End,Peak,Region
<fct>,<int>,<int>,<chr>,<chr>
chr8,126778902,126779728,chr8_126778902_126779728,MYC
chr8,126782925,126783318,chr8_126782925_126783318,MYC
chr8,126804776,126805825,chr8_126804776_126805825,MYC
chr8,126817018,126817490,chr8_126817018_126817490,MYC
chr8,126820465,126821393,chr8_126820465_126821393,MYC
chr8,126824093,126824987,chr8_126824093_126824987,MYC


In [17]:
tail(dat)

Chrom,Start,End,Peak,Region
<fct>,<int>,<int>,<chr>,<chr>
chrX,49840468,49841285,chrX_49840468_49841285,GATA1
chrX,49856806,49857279,chrX_49856806_49857279,GATA1
chrX,49860981,49861475,chrX_49860981_49861475,GATA1
chrX,49862119,49862823,chrX_49862119_49862823,GATA1
chrX,49868244,49869175,chrX_49868244_49869175,GATA1
chrX,49878402,49880117,chrX_49878402_49880117,GATA1


In [18]:
fdiry = file.path(FD_RES, "region", "KS91_K562_ASTARRseq_peak_macs")
fname = "KS91_K562_hg38_ASTARRseq_Input.peaks.MYC.FADS.GATA1.tsv"
fpath = file.path(fdiry, fname)

write_tsv(dat_peak, fpath)

In [19]:
fdiry = file.path(FD_RES, "region", "KS91_K562_ASTARRseq_peak_macs")
fname = "KS91_K562_hg38_ASTARRseq_Input.peaks.MYC.FADS.GATA1.bed"
fpath = file.path(fdiry, fname)

write_tsv(dat_peak, fpath, col_names = FALSE)